# Data Science Interview Simulation: Practical Scenarios
**Duration: ~60 minutes**

This notebook simulates a technical interview environment with realistic data scenarios. You'll work through progressively challenging problems that test your SQL, data modeling, Python, and analytical skills.

## Instructions
- Read each scenario carefully
- Write your solution in the provided code cells
- Test your code to ensure it works
- Be prepared to explain your approach

---

## Setup: Museum Database

You're interviewing for a data analyst position at a museum that tracks visitors, exhibitions, and artwork. First, let's set up the database.

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta

# Create database connection
conn = sqlite3.connect('museum_interview.db')
cursor = conn.cursor()

print("Database connection established")

In [ ]:
# Create the museum database schema

# Artists table
cursor.execute('''
CREATE TABLE IF NOT EXISTS artists (
    artist_id INTEGER PRIMARY KEY,
    artist_name TEXT NOT NULL,
    birth_year INTEGER,
    nationality TEXT,
    movement TEXT
)
''')

# Artworks table
cursor.execute('''
CREATE TABLE IF NOT EXISTS artworks (
    artwork_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    artist_id INTEGER,
    creation_year INTEGER,
    medium TEXT,
    value_usd DECIMAL(12, 2),
    FOREIGN KEY (artist_id) REFERENCES artists(artist_id)
)
''')

# Exhibitions table
cursor.execute('''
CREATE TABLE IF NOT EXISTS exhibitions (
    exhibition_id INTEGER PRIMARY KEY,
    exhibition_name TEXT NOT NULL,
    start_date DATE,
    end_date DATE,
    gallery TEXT,
    ticket_price DECIMAL(6, 2)
)
''')

# Exhibition_Artworks (many-to-many relationship)
cursor.execute('''
CREATE TABLE IF NOT EXISTS exhibition_artworks (
    exhibition_id INTEGER,
    artwork_id INTEGER,
    PRIMARY KEY (exhibition_id, artwork_id),
    FOREIGN KEY (exhibition_id) REFERENCES exhibitions(exhibition_id),
    FOREIGN KEY (artwork_id) REFERENCES artworks(artwork_id)
)
''')

# Visitors table
cursor.execute('''
CREATE TABLE IF NOT EXISTS visitors (
    visitor_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    email TEXT,
    membership_level TEXT
)
''')

# Visits table
cursor.execute('''
CREATE TABLE IF NOT EXISTS visits (
    visit_id INTEGER PRIMARY KEY,
    visitor_id INTEGER,
    exhibition_id INTEGER,
    visit_date DATE,
    ticket_price_paid DECIMAL(6, 2),
    duration_minutes INTEGER,
    FOREIGN KEY (visitor_id) REFERENCES visitors(visitor_id),
    FOREIGN KEY (exhibition_id) REFERENCES exhibitions(exhibition_id)
)
''')

conn.commit()
print("Schema created successfully")

In [ ]:
# Insert sample data

# Artists
artists_data = [
    (1, 'Claude Monet', 1840, 'French', 'Impressionism'),
    (2, 'Vincent van Gogh', 1853, 'Dutch', 'Post-Impressionism'),
    (3, 'Pablo Picasso', 1881, 'Spanish', 'Cubism'),
    (4, 'Frida Kahlo', 1907, 'Mexican', 'Surrealism'),
    (5, 'Leonardo da Vinci', 1452, 'Italian', 'Renaissance'),
    (6, 'Rembrandt', 1606, 'Dutch', 'Baroque')
]
cursor.executemany('INSERT OR IGNORE INTO artists VALUES (?, ?, ?, ?, ?)', artists_data)

# Artworks
artworks_data = [
    (1, 'Water Lilies', 1, 1906, 'Oil on canvas', 40000000),
    (2, 'Impression, Sunrise', 1, 1872, 'Oil on canvas', 35000000),
    (3, 'The Starry Night', 2, 1889, 'Oil on canvas', 100000000),
    (4, 'Sunflowers', 2, 1888, 'Oil on canvas', 75000000),
    (5, 'Les Demoiselles d\'Avignon', 3, 1907, 'Oil on canvas', 90000000),
    (6, 'Guernica', 3, 1937, 'Oil on canvas', 200000000),
    (7, 'The Two Fridas', 4, 1939, 'Oil on canvas', 50000000),
    (8, 'Self-Portrait with Thorn Necklace', 4, 1940, 'Oil on canvas', 45000000),
    (9, 'Mona Lisa', 5, 1503, 'Oil on panel', 850000000),
    (10, 'The Last Supper', 5, 1498, 'Tempera', 600000000),
    (11, 'The Night Watch', 6, 1642, 'Oil on canvas', 75000000)
]
cursor.executemany('INSERT OR IGNORE INTO artworks VALUES (?, ?, ?, ?, ?, ?)', artworks_data)

# Exhibitions
exhibitions_data = [
    (1, 'Impressionist Masters', '2024-01-15', '2024-03-31', 'Main Gallery', 25.00),
    (2, 'Modern Art Revolution', '2024-02-01', '2024-04-30', 'East Wing', 30.00),
    (3, 'Renaissance Wonders', '2024-03-15', '2024-06-15', 'Grand Hall', 35.00),
    (4, 'Dutch Golden Age', '2024-04-01', '2024-05-31', 'North Gallery', 20.00)
]
cursor.executemany('INSERT OR IGNORE INTO exhibitions VALUES (?, ?, ?, ?, ?, ?)', exhibitions_data)

# Exhibition-Artworks relationships
exhibition_artworks_data = [
    (1, 1), (1, 2), (1, 3), (1, 4),  # Impressionist exhibition
    (2, 5), (2, 6), (2, 7), (2, 8),  # Modern Art exhibition
    (3, 9), (3, 10),  # Renaissance exhibition
    (4, 11)  # Dutch exhibition
]
cursor.executemany('INSERT OR IGNORE INTO exhibition_artworks VALUES (?, ?)', exhibition_artworks_data)

# Visitors
visitors_data = [
    (1, 'Sarah', 'Johnson', 'sarah.j@email.com', 'Premium'),
    (2, 'Michael', 'Chen', 'mchen@email.com', 'Standard'),
    (3, 'Emily', 'Rodriguez', 'emily.r@email.com', 'Premium'),
    (4, 'David', 'Kim', 'dkim@email.com', 'Standard'),
    (5, 'Jessica', 'Patel', 'jpatel@email.com', 'Basic'),
    (6, 'Robert', 'Taylor', 'rtaylor@email.com', 'Premium'),
    (7, 'Maria', 'Garcia', 'mgarcia@email.com', 'Standard'),
    (8, 'James', 'Wilson', 'jwilson@email.com', 'Basic')
]
cursor.executemany('INSERT OR IGNORE INTO visitors VALUES (?, ?, ?, ?, ?)', visitors_data)

# Visits
visits_data = [
    (1, 1, 1, '2024-01-20', 25.00, 120),
    (2, 1, 2, '2024-02-15', 30.00, 90),
    (3, 2, 1, '2024-01-25', 25.00, 75),
    (4, 3, 1, '2024-02-01', 25.00, 110),
    (5, 3, 3, '2024-03-20', 35.00, 150),
    (6, 4, 2, '2024-02-10', 30.00, 60),
    (7, 5, 1, '2024-01-30', 25.00, 45),
    (8, 6, 3, '2024-03-25', 35.00, 180),
    (9, 7, 2, '2024-02-20', 30.00, 85),
    (10, 8, 1, '2024-02-05', 25.00, 50),
    (11, 2, 4, '2024-04-15', 20.00, 70),
    (12, 4, 4, '2024-04-20', 20.00, 65)
]
cursor.executemany('INSERT OR IGNORE INTO visits VALUES (?, ?, ?, ?, ?, ?)', visits_data)

conn.commit()
print("Sample data inserted successfully")
print(f"\nArtists: {len(artists_data)}")
print(f"Artworks: {len(artworks_data)}")
print(f"Exhibitions: {len(exhibitions_data)}")
print(f"Visitors: {len(visitors_data)}")
print(f"Visits: {len(visits_data)}")

---
## Problem 1: Data Exploration (10 minutes)

**Scenario**: The museum director wants to understand the current state of the database.

### Task 1.1: Write a query to show all exhibitions with their date ranges and ticket prices

In [ ]:
# YOUR CODE HERE
query_1_1 = """
-- Write your SQL query here

"""

# Test your query
# df_exhibitions = pd.read_sql_query(query_1_1, conn)
# print(df_exhibitions)

### Task 1.2: Find all artworks by Impressionist or Post-Impressionist artists

Include the artist name, artwork title, and creation year.

In [ ]:
# YOUR CODE HERE
query_1_2 = """
-- Write your SQL query here

"""

# Test your query
# df_impressionist = pd.read_sql_query(query_1_2, conn)
# print(df_impressionist)

---
## Problem 2: Revenue Analysis (15 minutes)

**Scenario**: The CFO wants to analyze exhibition revenue and visitor patterns.

### Task 2.1: Calculate total revenue and visitor count for each exhibition

Your result should include:
- Exhibition name
- Total number of visits
- Total revenue
- Average visit duration

Order by total revenue (highest first).

In [ ]:
# YOUR CODE HERE
query_2_1 = """
-- Write your SQL query here

"""

# Test your query
# df_revenue = pd.read_sql_query(query_2_1, conn)
# print(df_revenue)

### Task 2.2: Identify high-value visitors

Create a query to find visitors who have:
- Made more than 2 visits
- Spent more than $50 total

Include their name, membership level, total visits, and total spent.

In [ ]:
# YOUR CODE HERE
query_2_2 = """
-- Write your SQL query here

"""

# Test your query
# df_high_value = pd.read_sql_query(query_2_2, conn)
# print(df_high_value)

---
## Problem 3: Complex Analysis (15 minutes)

**Scenario**: The museum wants to optimize their exhibition strategy.

### Task 3.1: Calculate the value-to-visitors ratio

For each exhibition, calculate:
- Total value of artworks displayed (sum of artwork values)
- Number of unique visitors
- Value per visitor (total artwork value / number of visitors)

This helps understand which exhibitions attract visitors relative to their collection value.

In [ ]:
# YOUR CODE HERE
query_3_1 = """
-- Write your SQL query here
-- Hint: You'll need to join multiple tables and use aggregations

"""

# Test your query
# df_value_analysis = pd.read_sql_query(query_3_1, conn)
# print(df_value_analysis)

### Task 3.2: Find the most popular artist

Determine which artist's works have been viewed the most (total visits to exhibitions featuring their work).

Show:
- Artist name
- Number of their artworks in exhibitions
- Total visits to exhibitions featuring their work
- Average visit duration for exhibitions featuring their work

In [ ]:
# YOUR CODE HERE
query_3_2 = """
-- Write your SQL query here
-- Hint: This requires joining artists -> artworks -> exhibition_artworks -> exhibitions -> visits

"""

# Test your query
# df_popular_artists = pd.read_sql_query(query_3_2, conn)
# print(df_popular_artists)

---
## Problem 4: Python Data Analysis (15 minutes)

**Scenario**: The marketing team wants insights that require more complex analysis than SQL can easily provide.

### Task 4.1: Visitor Segmentation Analysis

Using pandas, analyze visitor behavior by membership level:
1. Load all visits with visitor information
2. Calculate for each membership level:
   - Average spending per visit
   - Average visit duration
   - Number of unique visitors
   - Total revenue contribution
3. Create a summary visualization or table

In [ ]:
# YOUR CODE HERE
# Step 1: Load the data
query = """
SELECT
    v.visit_id,
    v.visitor_id,
    vis.membership_level,
    v.ticket_price_paid,
    v.duration_minutes
FROM visits v
JOIN visitors vis ON v.visitor_id = vis.visitor_id
"""

df_visits = pd.read_sql_query(query, conn)

# Step 2: Perform your analysis
# YOUR CODE HERE


# Step 3: Display results
# YOUR CODE HERE


### Task 4.2: Time-based Trend Analysis

Create a time series analysis showing:
1. Daily visitor count
2. 7-day rolling average of visitors
3. Identify the busiest day of the week

**Hint**: You may need to parse dates and use pandas datetime functionality.

In [ ]:
# YOUR CODE HERE
# Load visit dates
query = """
SELECT
    visit_date,
    COUNT(*) as visit_count
FROM visits
GROUP BY visit_date
ORDER BY visit_date
"""

df_daily = pd.read_sql_query(query, conn)

# Convert to datetime and perform analysis
# YOUR CODE HERE


---
## Problem 5: Database Design Challenge (15 minutes)

**Scenario**: The museum wants to expand their system to include:
- Audio guides (which visitors can rent)
- Feedback/ratings for exhibitions
- Gift shop purchases

### Task 5.1: Design the Schema

Create an ERD (Entity-Relationship Diagram) description or SQL schema for:

1. **Audio Guides**: Track which visitors rent audio guides, rental fee, return status
2. **Exhibition Ratings**: Allow visitors to rate exhibitions (1-5 stars) with optional comments
3. **Gift Shop**: Track items, prices, and purchases by visitors

Write the CREATE TABLE statements with appropriate:
- Primary keys
- Foreign keys
- Data types
- Constraints

In [ ]:
# YOUR CODE HERE
# Write CREATE TABLE statements for the three new features

# Audio Guides table
audio_guides_schema = """
CREATE TABLE audio_guide_rentals (
    -- YOUR CODE HERE
);
"""

# Exhibition Ratings table
ratings_schema = """
CREATE TABLE exhibition_ratings (
    -- YOUR CODE HERE
);
"""

# Gift Shop tables (you might need multiple tables)
gift_shop_schema = """
CREATE TABLE gift_shop_items (
    -- YOUR CODE HERE
);

CREATE TABLE gift_shop_purchases (
    -- YOUR CODE HERE
);
"""

# Print your schemas
print("Audio Guides Schema:")
print(audio_guides_schema)
print("\nRatings Schema:")
print(ratings_schema)
print("\nGift Shop Schema:")
print(gift_shop_schema)

### Task 5.2: Explain Your Design Decisions

In the cell below, explain:
1. Why you chose specific data types
2. How you handled relationships between tables
3. Any assumptions you made
4. How your design supports common queries

**YOUR EXPLANATION HERE:**

(Write your design justification here)

---
## Bonus Challenge: Advanced Analytics (Optional)

**Scenario**: The museum wants predictive insights.

### Bonus Task: Revenue Forecasting

Based on the current data:
1. Calculate the average revenue per exhibition per day
2. Identify seasonal or temporal patterns
3. Predict potential revenue for a new 60-day exhibition
4. Recommend optimal pricing based on historical data

Use both SQL and Python for this analysis.

In [ ]:
# YOUR CODE HERE
# This is an open-ended problem - show your analytical thinking!


---
## Problem 6: Data Quality & Cleaning (10 minutes)

**Scenario**: You discover some data quality issues in the database.

### Task 6.1: Identify Data Quality Issues

Write queries to find:
1. Any visits where ticket_price_paid doesn't match the exhibition's ticket_price
2. Artworks without an associated artist
3. Exhibitions with no artwork assigned
4. Any duplicate visitor emails

In [ ]:
# YOUR CODE HERE
# Write queries for each data quality check

# 1. Price mismatches
query_price_check = """
-- YOUR QUERY HERE
"""

# 2. Orphaned artworks
query_orphaned_artworks = """
-- YOUR QUERY HERE
"""

# 3. Empty exhibitions
query_empty_exhibitions = """
-- YOUR QUERY HERE
"""

# 4. Duplicate emails
query_duplicate_emails = """
-- YOUR QUERY HERE
"""

# Run your checks
# print("Price Mismatches:")
# print(pd.read_sql_query(query_price_check, conn))
# ... etc for other checks

---
## Reflection Questions

After completing the exercises, consider these interview discussion points:

1. **Performance Optimization**:
   - Which queries might be slow with millions of records?
   - What indexes would you create?
   - How would you optimize the slowest query?

2. **Scalability**:
   - How would you modify this design for multiple museums?
   - What if visitor data grows to 10 million records?
   - How would you handle real-time analytics?

3. **Data Governance**:
   - What privacy concerns exist with visitor data?
   - How would you implement data retention policies?
   - What audit trails might be necessary?

4. **Business Intelligence**:
   - What KPIs would you track?
   - How would you build a dashboard?
   - What predictive models might be useful?

5. **Technical Decisions**:
   - Why use a relational database vs. NoSQL for this application?
   - When would you denormalize for performance?
   - How would you handle concurrent updates?

Be prepared to discuss these in your interview!

---
## Solutions Reference

After attempting all problems, you can check your approach against best practices:

### Key Learning Points:

1. **SQL Mastery**:
   - Comfortable with multi-table JOINs
   - Use of aggregate functions with GROUP BY
   - Subqueries and CTEs for complex logic
   - Window functions for ranking and analytics

2. **Data Modeling**:
   - Proper use of primary and foreign keys
   - Understanding of many-to-many relationships
   - Normalization principles
   - Constraint design

3. **Python/Pandas**:
   - Loading data from SQL
   - GroupBy operations
   - DateTime handling
   - Data cleaning techniques

4. **Analytical Thinking**:
   - Breaking down complex problems
   - Choosing appropriate tools (SQL vs. Python)
   - Performance considerations
   - Business context awareness

---

**Good luck with your interview!**

In [ ]:
# Clean up
conn.close()
print("Database connection closed")